In [ ]:
from PIL import Image
import numpy as np
import math
import typing
import random

In [ ]:
def initialize_2dimage(dimensions: tuple, background_color: tuple) -> list:
    return [[background_color for _ in range(dimensions[1])] for _ in range(dimensions[0])]

In [ ]:
def line_2d(point_1: tuple, point_2: tuple):
    x1, y1 = point_1
    x2, y2 = point_2
    assert not (x1%1 and x2%1 and y1%1 and y2%1), f"input points must whole numbers not decimals: input : {x1},{x2},{y1},{y2} "
    difference_x = abs(x2 - x1)
    difference_y = abs(y2 - y1)
    if x2 > x1:
        x_sign = 1
    else:
        x_sign = -1
    if y2 > y1:
        y_sign = 1
    else:
        y_sign = -1
    if max(difference_x,difference_y) == difference_x:
        y_error = 2*difference_y - difference_x
        yield (int(x1),int(y1))
        while x1 != x2:
            x1 += x_sign
            if y_error >= 0:
                y1 += y_sign
                y_error -= 2 * difference_x
            y_error += 2 * difference_y 
            yield (int(x1),int(y1))
    elif max(difference_x,difference_y) == difference_y:
        x_error = 2*difference_x - difference_y
        yield (int(x1),int(y1))
        while y1 != y2:
            y1 += y_sign
            if x_error >= 0:
                x1 += x_sign
                x_error -= 2 * difference_y
            x_error += 2 * difference_x 
            yield (int(x1),int(y1))

In [ ]:
def save_image(pixels: list, image_name: str):
    pixels = list(zip(*pixels))[::-1]
    array = np.array(pixels, dtype=np.uint8)
    new_image = Image.fromarray(array)
    display(new_image)
    new_image.save(image_name + '.png')

In [ ]:
class camera:
    def __init__(self, fov: int, vertical_fov: int, resolution: tuple, location: tuple, direction: tuple, view_distance: tuple):
        self.fov = fov
        self.vertical_fov = vertical_fov
        self.resolution = resolution
        self.location = location
        self.direction = direction
        self.view_distance = view_distance

In [ ]:
class mesh3D:
    def __init__(self, verticies: list, faces: list, center: tuple):
        self.verticies = verticies
        self.faces = faces
        self.center = center